# Basic Session-Scoped UDTF Registration

This notebook demonstrates how to register UDTFs for session-scoped use in Databricks.

## Prerequisites

- Databricks workspace with active Spark session
- CDF credentials in `config.toml` file
- CDF Data Model with Views


## Step 1: Install Dependencies


In [ ]:
# Install required packages
# Note: Restart Python kernel after installation when prompted

%pip install cognite-sdk cognite-databricks


## Step 2: Load CDF Client


In [ ]:
from databricks.sdk import WorkspaceClient
from cognite.pygen import load_cognite_client_from_toml
from cognite.client.data_classes.data_modeling.ids import DataModelId

# Create WorkspaceClient (auto-detects credentials in Databricks)
workspace_client = WorkspaceClient()

# Load client from TOML file
# config.toml format:
# [cognite]
# project = "<cdf-project>"
# tenant_id = "<tenant-id>"
# cdf_cluster = "<cdf-cluster>"
# client_id = "<client-id>"
# client_secret = "<client-secret>"

client = load_cognite_client_from_toml("config.toml")
print("✓ CDF client loaded")


## Step 3: Generate UDTFs


In [ ]:
from cognite.databricks import generate_udtf_notebook

# Define data model
data_model_id = DataModelId(
    space="sailboat",
    external_id="sailboat",
    version="v1"
)

# Generate UDTFs with all parameters
generator = generate_udtf_notebook(
    data_model_id,
    client,
    workspace_client=workspace_client,  # Include this for full functionality
    output_dir="/Workspace/Users/user@example.com/udtf",
    # Note: catalog and schema parameters are only used for Unity Catalog registration,
    # not for session-scoped UDTFs. They can be omitted for session-scoped use.
)

print("✓ UDTFs generated (includes time series UDTFs)")


## Step 4: Register UDTFs for Session-Scoped Use


In [ ]:
# Register all UDTFs for session-scoped use (includes time series UDTFs automatically)
registered = generator.register_session_scoped_udtfs()

# Print registered functions
print("\n✓ Registered UDTFs (includes time series UDTFs):")
for view_id, func_name in registered.items():
    print(f"  - {view_id} -> {func_name}")


## Step 5: Query UDTF with Named Parameters (SQL)


In [ ]:
# Example: Query a registered UDTF using named parameters (recommended)
# Replace 'small_boat_udtf' with your actual function name from Step 4
# Replace 'cdf_sailboat_sailboat' with your secret scope name

query = """
SELECT * FROM small_boat_udtf(
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project'),
    name => 'MyBoat',
    description => NULL
) LIMIT 10;
"""

# Execute query
result = spark.sql(query)
result.show()
